<a href="https://colab.research.google.com/github/Davioliveira1305/Metodos-Numericos-2/blob/main/Tarefa7.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Funções Hiperbólicas

In [1]:
import numpy as np

# Hiperbólica seno
def sinh(x):
    return (np.exp(x) - np.exp(-x)) / 2

# Hiperbólica cosseno
def cosh(x):
    return (np.exp(x) + np.exp(-x)) / 2

# Hiperbólica tangente
def tanh(x):
    return sinh(x) / cosh(x)


# Fórmulas de Newton-Cotes

In [3]:
def pol_inter_closed(f,points, div, degree):
  # Intervalo
  h = (points[-1] - points[0])/div
  if degree == 1: return h*(f(points[0]) + f(points[1]))
  elif degree == 2: return h*(f(points[0]) + 4 * f(points[1]) + f(points[2]))
  elif degree == 3: return h*(f(points[0]) + 3 * f(points[1]) + 3 * f(points[2]) + f(points[3]))
  else: return h*(7 * f(points[0]) + 32 * f(points[1]) + 12 * f(points[2]) + 32 * f(points[3]) + 7 * f(points[4]))

import warnings, numba
import numpy

warnings.filterwarnings('ignore')
# Lib para acelerar códigos python
@numba.jit
def newton_cotes_closed(f, a, b, degree):
  # Inicializando o erro em 10
  E = 10
  # Vetor para guardar o intervalo de integração
  interval = [a, b]
  # Inicializando com o número de partições igual a 1
  N = 1
  # Definindo do termo que vamos dividir o intervalo h do polinômio interpolador
  if degree == 1: div = 2
  elif degree == 2: div = 6
  elif degree == 3: div = 8
  elif degree == 4: div = 90
  # Inicializando da variável que vai guardas as soluções antigas
  Sol_old = 0
  # Inicializando da variável que vai guardas as soluções novas
  Sol_new = 0
  while E > 1e-6:
    Sol_new = 0
    # Particiona o intervalo de integração em relação ao número de partições
    partitions = numpy.linspace(interval[0], interval[1], N + 1)
    # Pecorre as partições
    for i in range(len(partitions) - 1):
        a = partitions[i+0]
        b = partitions[i+1]
        # Retorna os pontos que serão utilizados no polinômio interpolador
        f_points = numpy.linspace(a, b, degree + 1)
        # Soma o resulta da interpolação nas diversas partições
        Sol_new += pol_inter_closed(f, f_points,div,degree)
    # Aumenta o número de partições
    N = N * 2
    # Calcula o erro relativos da solução
    E = abs(1 - Sol_old/Sol_new)
    # Atualiza a solução antiga
    Sol_old = Sol_new
  return Sol_new, N, E

# Funções que queremos integrar:
$$\int^{1}_{-1}\frac{dx}{\sqrt[3]{x^{2}}}$$
$$\int^{0}_{-2} \frac{dx}{\sqrt{4 - x^{2}}}$$


## Intervalo de Integração da primeira função.

In [60]:
# O intervalo foi modificado, pois, a singularidade encontra-se no ponto x = 0
a = 0
b = 1

## Função que será integrada

In [61]:
# A função foi modificado para suportar o novo intervalo de intervalo
def func(x):
  return 2 * (1/(((x ** 2))**(1/3)))

# Exponencial Simples


## Mudança de variável, utilizando a estratégia de exponencial simples

In [69]:
def mud_variavel(s, a, b):
  return ((a + b)/2) + ((b - a)/2)*tanh(s)

def exp_simples(s, a = a, b = b):
  # Mudança de variável
  x_s = mud_variavel(s, a, b)
  # Derivada em relação a s
  deri = ((b - a)/2) * (1/((cosh(s))**2))
  return func(x_s) * deri


# Exponencial Dupla

## Mudança de variável, utilizando a estratégia de exponencial dupla

In [70]:
def mud_variavel_d(s, a, b):
  return ((a + b)/2) + ((b - a)/2)*tanh((math.pi/2)*sinh(s))

def exp_dupla(s, a = a, b = b):
  # Mudança de variável
  x_s = mud_variavel_d(s, a, b)
  # Derivada em relação a s
  deri = ((b - a)/2) * ((math.pi/2) * ((cosh(s))/((cosh((math.pi/2)*sinh(s)))**2)))
  return func(x_s) * deri

## Definição do melhor intervalo de integração depois da mudança de variável

In [71]:
import numpy
import math
def convergencia(simples = True):
  c = 1
  new_result = 0
  old_result = 0
  while True:
    if simples:
      new_result = newton_cotes_closed(exp_simples, -c, c, 4)[0]
      E = abs((new_result - old_result)/new_result)
      if np.isnan(new_result) or math.isinf(new_result): break
      old_result = new_result
      if E < 1e-5: break
      c = c + 0.1
      print(f'Solução = {new_result}, intervalo de integração = [{-c}, {c}]')
    else:
      new_result = newton_cotes_closed(exp_dupla, -c, c, 4)[0]
      E = abs((new_result - old_result)/new_result)
      if np.isnan(new_result) or math.isinf(new_result): break
      old_result = new_result
      if E < 1e-5: break
      c = c + 0.1
      print(f'Solução = {new_result}, intervalo de integração = [{-c}, {c}]')

# Teste da primeira função

## Teste utilizando a estratégia da exponencial simples

In [65]:
convergencia(True)

Solução = 2.7985518930487463, intervalo de integração = [-1.1, 1.1]
Solução = 3.0108364764540716, intervalo de integração = [-1.2000000000000002, 1.2000000000000002]
Solução = 3.2097607673707924, intervalo de integração = [-1.3000000000000003, 1.3000000000000003]
Solução = 3.3958239869139977, intervalo de integração = [-1.4000000000000004, 1.4000000000000004]
Solução = 3.5696184779044007, intervalo de integração = [-1.5000000000000004, 1.5000000000000004]
Solução = 3.7317919818323664, intervalo de integração = [-1.6000000000000005, 1.6000000000000005]
Solução = 3.883018229611651, intervalo de integração = [-1.7000000000000006, 1.7000000000000006]
Solução = 4.02397491635241, intervalo de integração = [-1.8000000000000007, 1.8000000000000007]
Solução = 4.155327948131001, intervalo de integração = [-1.9000000000000008, 1.9000000000000008]
Solução = 4.277720581403185, intervalo de integração = [-2.000000000000001, 2.000000000000001]
Solução = 4.391766552854023, intervalo de integração = [-

## Teste utilizando a estratégia da exponencial dupla

In [66]:
convergencia(False)

Solução = 4.212705714625484, intervalo de integração = [-1.1, 1.1]
Solução = 4.4960239873208225, intervalo de integração = [-1.2000000000000002, 1.2000000000000002]
Solução = 4.7512487839368625, intervalo de integração = [-1.3000000000000003, 1.3000000000000003]
Solução = 4.978713563051258, intervalo de integração = [-1.4000000000000004, 1.4000000000000004]
Solução = 5.178897731421507, intervalo de integração = [-1.5000000000000004, 1.5000000000000004]
Solução = 5.352466225240022, intervalo de integração = [-1.6000000000000005, 1.6000000000000005]
Solução = 5.500337113147852, intervalo de integração = [-1.7000000000000006, 1.7000000000000006]
Solução = 5.6237549406682765, intervalo de integração = [-1.8000000000000007, 1.8000000000000007]
Solução = 5.724345514263671, intervalo de integração = [-1.9000000000000008, 1.9000000000000008]
Solução = 5.804131031628891, intervalo de integração = [-2.000000000000001, 2.000000000000001]
Solução = 5.865489920165714, intervalo de integração = [-2.

# Teste da segunda função

In [67]:
# Definição da segunda função
def func(x): return 1/(math.sqrt(4 - x**2))

### Intervalo de integração da segunda função

In [68]:
a = -2
b = 0

## Teste utilizando a estratégia da exponencial simples

In [72]:
convergencia(True)

Solução = 0.9580559292785322, intervalo de integração = [-1.1, 1.1]
Solução = 1.0204256160865033, intervalo de integração = [-1.2000000000000002, 1.2000000000000002]
Solução = 1.0767917036169155, intervalo de integração = [-1.3000000000000003, 1.3000000000000003]
Solução = 1.1275702772362228, intervalo de integração = [-1.4000000000000004, 1.4000000000000004]
Solução = 1.1732043779239163, intervalo de integração = [-1.5000000000000004, 1.5000000000000004]
Solução = 1.214142195657735, intervalo de integração = [-1.6000000000000005, 1.6000000000000005]
Solução = 1.2508214174044143, intervalo de integração = [-1.7000000000000006, 1.7000000000000006]
Solução = 1.2836587868951808, intervalo de integração = [-1.8000000000000007, 1.8000000000000007]
Solução = 1.3130437056727222, intervalo de integração = [-1.9000000000000008, 1.9000000000000008]
Solução = 1.339334979506111, intervalo de integração = [-2.000000000000001, 2.000000000000001]
Solução = 1.362859746552756, intervalo de integração =

## Teste utilizando a estratégia da exponencial dupla

In [73]:
convergencia(False)

Solução = 1.3255018698912961, intervalo de integração = [-1.1, 1.1]
Solução = 1.3835212994200594, intervalo de integração = [-1.2000000000000002, 1.2000000000000002]
Solução = 1.4305664322017584, intervalo de integração = [-1.3000000000000003, 1.3000000000000003]
Solução = 1.468047028239581, intervalo de integração = [-1.4000000000000004, 1.4000000000000004]
Solução = 1.4973278595036303, intervalo de integração = [-1.5000000000000004, 1.5000000000000004]
Solução = 1.5196973772247175, intervalo de integração = [-1.6000000000000005, 1.6000000000000005]
Solução = 1.5363518500028366, intervalo de integração = [-1.7000000000000006, 1.7000000000000006]
Solução = 1.5483865286562484, intervalo de integração = [-1.8000000000000007, 1.8000000000000007]
Solução = 1.5567878358604346, intervalo de integração = [-1.9000000000000008, 1.9000000000000008]
Solução = 1.5624246366035688, intervalo de integração = [-2.000000000000001, 2.000000000000001]
Solução = 1.5660392421137421, intervalo de integração